In [1]:
%run src_runs/autorun/daily_update.py

Use device name: NVIDIA GeForce RTX 4090
update data: ********************
Unpack Update Files
Sat Nov 16 13:39:25 2024 : download since!
Connection and Factor preparation finished!
Sat Nov 16 13:39:25 2024 : All Updates Done! Cost 0.50 Secs
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
process other min bars:
--------------------------------------------------------------------------------
process other min bars:
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
prepare predict data: ********************
predict is True , Data Processing start!
6 datas :['y', 'day', '30m', 'style', 'indus', 'week']
y is up to 20241116 already!
day is up to 20241116 already!
30m is up to 20241116 alr

100%|██████████| 333/333 [00:00<00:00, 3737.91it/s]


--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
update_factors: ********************
Finish model [gru_day] predicting!
Finish model [gruRTN_day] predicting!
Finish model [gru_avg] predicting!
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
sending email success


In [1]:
from src.api import DataAPI , ModelAPI

#print('update_data: ' + '*' * 20)
DataAPI.update()

ModelAPI.update()


Basic module imported!
INSTANCE_RECORD can be accessed to check trainer , account or else...
--------------------------------------------------------------------------------
******************************** DOWNLOAD DATA ********************************
DataBase object [trade_ts],[day_limit],[20241126] save to D:\Coding\learndl\learndl\data\DataBase\DB_trade_ts\day_limit\2024\day_limit.20241126.feather successfully
DataBase object [trade_ts],[day_moneyflow],[20241126] save to D:\Coding\learndl\learndl\data\DataBase\DB_trade_ts\day_moneyflow\2024\day_moneyflow.20241126.feather successfully
DataBase object [trade_ts],[day],[20241126] save to D:\Coding\learndl\learndl\data\DataBase\DB_trade_ts\day\2024\day.20241126.feather successfully
DataBase object [trade_ts],[day_val],[20241126] save to D:\Coding\learndl\learndl\data\DataBase\DB_trade_ts\day_val\2024\day_val.20241126.feather successfully


c:\Users\jinmeng\AppData\Local\Programs\Python\Python311\Lib\site-packages\rqdatac\client.py:257: UserWarning: Your account will be expired after  9 days. Please call us at 0755-22676337 to upgrade or purchase or renew your contract.
  warnings.warn("Your account will be expired after  {} days. "


DataBase object [trade_ts],[min],[20241126] save to D:\Coding\learndl\learndl\data\DataBase\DB_trade_ts\min\2024\min.20241126.feather successfully
rcquant bar min 20241126 success
--------------------------------------------------------------------------------
process other min bars at 20241126 from source rcquant
DataBase object [trade_ts],in],[20241126] save to D:\Coding\learndl\learndl\data\DataBase\DB_trade_ts\5min\2024\5min.20241126.feather successfully
DataBase object [trade_ts],in],[20241126] save to D:\Coding\learndl\learndl\data\DataBase\DB_trade_ts\10min\2024\10min.20241126.feather successfully
DataBase object [trade_ts],in],[20241126] save to D:\Coding\learndl\learndl\data\DataBase\DB_trade_ts\15min\2024\15min.20241126.feather successfully
DataBase object [trade_ts],in],[20241126] save to D:\Coding\learndl\learndl\data\DataBase\DB_trade_ts\30min\2024\30min.20241126.feather successfully
DataBase object [trade_ts],in],[20241126] save to D:\Coding\learndl\learndl\data\DataBase\

100%|██████████| 316/316 [00:00<00:00, 2661.73it/s]


Finish model [HiddenExtractingModel(hidden_name=gru_day_V0,name=gru_day,submodels=['best'],nums=None)] extracting!
--------------------------------------------------------------------------------
********************************************************************************
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
******************************** UPDATE FACTORS ********************************
model_name is None, predict all registered models
  -->  update pred for RegisteredModel(pred_name=gru_day_V0,name=gru_day,submodel=swalast,num=0)
  -->  update pred for RegisteredModel(pred_name=gruRTN_day_V0,name=gruRTN_day,submodel=swalast,num=0)
  -->  update pred for RegisteredModel(pred_name=gru_avg_V1,name=gru_avg,submodel=swalast,num=all)
********************************************************************************
--------------------------------------------------

ValueError: max() arg is an empty sequence

In [1]:
from src.api import ModelAPI
model = ModelAPI.Testor('ts_mixer')
model.try_forward()
model.try_metrics()

D:\Coding\learndl\learndl\configs\nn\ts_mixer.yaml does not exist, trying default.yaml
Load  2 DataBlocks...... finished! Cost 0.09 secs
Align 2 DataBlocks...... finished! Cost 0.23 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}


100%|██████████| 212/212 [00:00<?, ?it/s]


x shape is torch.Size([5169, 30, 6])
y shape is torch.Size([5169, 1])
Test Forward Success
x shape is torch.Size([5169, 30, 6])
y shape is torch.Size([5169, 1])
Test Forward Success
score function of [spearman] calculated and success!
loss function of [ccc] calculated and success!
metrics :  Metrics(loss=ccc,metric=spearman,penalty=['hidden_corr'])
Test Metrics Success


ModelTestor(model=NNPredictor(model_full_name=ts_mixer_day_ShortTest@0@0@0)) , check [.model][.batch_data][.metrics]